Separately run referrals and claims queries.
This file imports the results of those queries, cleans and standardizes each, and merges them for analysis.

In addition, we import clinical decision files and use it to "override" decisions that were made by analyzing the ROI for specialty & cpt code combination.

Finally, several outputs are prepared for reporting purposes, including the calculation of a projected AA approval rate.

In [ ]:
## Import required packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", 100)
import pyodbc
import os

## Import Data

#### Import data directly from SQL databases

Use referrals.sql, claims.sql, referrals_new.sql, cpt_desc.sql (saved in the same folder as this script) to pull data from relevants servers and databases.

In [ ]:
with open('referrals_quarterly.sql', 'r') as myfile:
    cpts_sql_str=myfile.read().replace('\n', ' ')

cnxn_cpts = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=IADS_V3')

cpts = pd.read_sql(cpts_sql_str, cnxn_cpts)

cnxn_cpts.close()

In [ ]:
with open('claims_quarterly.sql', 'r') as myfile:
    claims_sql_str=myfile.read().replace('\n', ' ')

cnxn_claims = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=NATIONAL_ANALYTICS')

claims = pd.read_sql(claims_sql_str, cnxn_claims)

cnxn_claims.close()

In [ ]:
with open('cpt_desc.sql', 'r') as myfile:
    cpt_desc_sql_str=myfile.read().replace('\n', ' ')

cnxn_cpt_desc = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=IADS_V3')

cpt_desc = pd.read_sql(cpt_desc_sql_str, cnxn_cpt_desc)

cnxn_cpt_desc.close()

In [ ]:
clinical_decisions_current = pd.read_excel('../data/AADictionary_Master.xlsx', sheet_name='Specialty Summary')

In [ ]:
#clinical_decisions_20190610 = pd.read_csv('../data/clinical_decision_20190610.csv')

In [ ]:
clinical_decisions_current = clinical_decisions_current[['Specialty', 'CPT_Code', 'is_PPL', 'Decision_Source',
       'Decision_Maker', 'Reason', 'Decision_Date', 'Decision']]

## Clean Referrals

A list of all specialties is used repeatedly in the code to loop through specialties and perform certain tasks.

In [ ]:
# A list of all specialties is used repeatedly in the code to loop through 
# specialties and perform certain tasks.
list_o_specs = cpts['Specialty'].unique().tolist()

In [ ]:
## Update date fields to conform to python datetime
cpts['Date_Decision'] = pd.to_datetime(cpts['Date_Decision'])
cpts['Date_Received'] = pd.to_datetime(cpts['Date_Received'])

In [ ]:
## Ensure that the max date from the data is recent, within the last three months
if (pd.datetime.now() - cpts['Date_Received'].max()) < timedelta(90):
    print("Data is near current and runs through {}".format(cpts['Date_Received'].max()))
else:
    print("Please update the referrals.sql file to pull more current data.")

In [ ]:
if (cpts['Date_Received'].max() - cpts['Date_Received'].min()) > timedelta(180):
    if (cpts['Date_Received'].max() - cpts['Date_Received'].min()) < timedelta(190):
        print("Referrals data contains 6 months of data, as expected.")
    else:
        print("Please update the referrals_quarterly.sql file to pull an entire year of data.")
else:
    print("Please update the referrals_quarterly.sql file to pull an entire year of data.")

In [ ]:
## Some specialties need to be broken into Senior and non-commercial 
## such that we can auto-approve for specific lines of business
new_lob = {'COMMERCIAL': '_not_sen',
               'SENIOR': '_senior',
          'MEDI-CAL': '_not_sen'}
cpts['LOB'] = cpts['LOB'].replace(new_lob)

In [ ]:
## Use this code to check whether recent changes have been captured in the rules
clinical_decisions_current[(clinical_decisions_current['Specialty']=='PAIN MANAGEMENT') &
                          (clinical_decisions_current['CPT_Code']=='64483')]

In [ ]:
## Define a list of specialties that will be broken out into LOB for the purposes of AA
specs_w_lob_distinct = ['RADIOLOGY']

In [ ]:
## Loop through specialties that should be broken by LOB and update the specialty column
for spec in list_o_specs:
    if spec in specs_w_lob_distinct:
        cpts['Specialty'] = np.where(cpts['Specialty']==spec, cpts['Specialty']+cpts['LOB'], cpts['Specialty'])
    

In [ ]:
# Now that some specialties are broken into senior and non-senior, need to recreate the list_o_specs
list_o_specs = cpts['Specialty'].unique().tolist()

In [ ]:
## flag retro statuses with 1 and 0 so they can be removed 
retro_conditions = [
 (cpts['status_name'] == 'APPROVED - RETRO REVIEW') |
 (cpts['status_name'] == 'DENIED - RETRO REVIEW') |
 (cpts['status_name'] == 'APPROVED - COB RETRO') |
 (cpts['status_name'] == 'PENDING - RETRO REVIEW') 
  ]

choices = [1]
cpts['is_retro'] = np.select(retro_conditions, choices, default=0)

## remove retros from list and drop 'is_retro' as it is no longer needed
cpts = cpts[cpts['is_retro']==0]
cpts.drop(columns='is_retro', inplace=True)

In [ ]:
## Claims data doesn't come with UNITS & we need to count the number of times a cpt code appears
## Here we re-write UNITS to 1.
cpts['UNITS'] = 1

In [ ]:
## Create an auto_approve flag
cpts['is_autoapp'] = np.where(cpts['status_name']=='APPROVED - AUTO', 1, 0)

In [ ]:
## If PPL field is null, assume the referral was not PPL
cpts.PPL.fillna("N", inplace=True)

In [ ]:
## Create a PPL flag
cpts['is_PPL'] = np.where(cpts['PPL']=='Y', 1, 0)

In [ ]:
## define list of status that should be considered denials
den_conditions = [
 (cpts['status_name'] == 'DENIED - CM') |
 (cpts['status_name'] == 'DENIED - BENEFIT CARVE OUT') |
 (cpts['status_name'] == 'DENIED - NOT A COVERED BENEFIT') |
 (cpts['status_name'] == 'DENIED - APPEAL') |
 (cpts['status_name'] == 'DENIED - CLINICAL TRIAL/EXP/INV') |
 (cpts['status_name'] == 'DENIED - TRANSPLANT') |
 (cpts['status_name'] == 'DENIED - MD') |
 (cpts['status_name'] == 'DENIED - CM/MD') |
 (cpts['status_name'] == 'DENIED - REDIRECT OSVN') |
 (cpts['status_name'] == 'DENIED - TICKLER')
  ]

## Create a denial flag
choices = [1]
cpts['is_den'] = np.select(den_conditions, choices, default=0)

In [ ]:
## Create a "approved" flag
cpts['is_app'] = np.where(cpts['status_cat']=='APPROVED', 1, 0)

## Create claims_sum

In [ ]:
## Some Specialty/CPT code combos appear in referrals data but not in claims data. 
## In those cases, we look in the claims data across all specialties to find an average
## Cost to be applied for that CPT Code.
claims_sum = claims.groupby(['CPT_Code'], as_index=False).agg({'avg_hcp_cost': 'mean'})

## Add variable to designate last 3 vs. prior 3

In [ ]:
## Split CPTs data into two parts, one for PRIOR_3 and one for LAST_3

In [ ]:
min_date = cpts['Date_Received'].min()

In [ ]:
max_date = cpts['Date_Received'].max()

In [ ]:
mid_date = min_date + (max_date - min_date)/2

In [ ]:
mid_date

In [ ]:
cpts['last3'] = np.where(cpts['Date_Received'] >= mid_date, 1, 0)

## Create cpts_manual

In [ ]:
## This is a cpt_code level list of all manually reviewed referrals

In [ ]:
cpts_manual = cpts[cpts['is_autoapp']==0]

In [ ]:
cpts_manual.pivot_table(values='HCP_CONNECT_AUTH_NUMBER', index=['is_autoapp', 'is_den'], aggfunc='count', margins=True)

In [ ]:
percent_of_year_in_data = (max_date - min_date).days / 365

In [ ]:
## Enter the total GA for the department for the year, divide by 2 since we're only looking at 1/2 of the year
dept_ga = 6500000 * percent_of_year_in_data

In [ ]:
## Calculate Cost per manually reviewed CPT code

ga_cpt = dept_ga / cpts[cpts['is_autoapp']==0].shape[0]

In [ ]:
ga_cpt

In [ ]:
## find the count of manually reviewed cpt codes from each specialty, cpt_code combo 
cpts_manual = cpts_manual.groupby(['last3', 'Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'UNITS' : 'count',
    'is_den': 'mean'
})

In [ ]:
## To differentiate the count of all units from manual units as we use both in a single
## file later
cpts_manual.rename(index=str, columns={'UNITS': 'UNITS_man'}, inplace=True)

In [ ]:
## Calculate the total cost of review any Specialty/cpt_code pair.
cpts_manual['cost_to_review'] = cpts_manual['UNITS_man']*ga_cpt

## Create cptssum

In [ ]:
## find the count, auto-approval rate, and denial rate from each specialty, cpt_code pair
cpts2 = cpts.groupby(['last3', 'Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'UNITS': 'count',
    'is_autoapp': 'mean',
})

In [ ]:
## Merge the cost to review (from CPT_manual) into cptssum
cpts3 = pd.merge(cpts2, cpts_manual, on=['last3', 'Specialty', 'CPT_Code', 'is_PPL'], how='left')

In [ ]:
## Merge average cost of a cpt code (from claims) into the referrals data
cpts4 = pd.merge(cpts3, claims, on=['Specialty', 'CPT_Code'], how='left')

In [ ]:
## For spec/cpt combos that don't have claims data associated, 
## use the average of that cpt across specialties
## NOTE: the average is not weighted, i.e. each specialties's average contributes equally to
## the applied average.
cpts_w_claims_fin = pd.merge(cpts4, claims_sum, on='CPT_Code', how='left')

In [ ]:
## If there is no average cost from claims at the spec/cpt pair level, fill it with the 
## average cost for the cpt ACROSS ALL SPECIALTIES
cpts_w_claims_fin['avg_hcp_cost_x'] = np.where(cpts_w_claims_fin['avg_hcp_cost_x'].isnull(), 
                                             cpts_w_claims_fin['avg_hcp_cost_y'],
                                             cpts_w_claims_fin['avg_hcp_cost_x'])

In [ ]:
## Drop unnecessary columns
cpts_w_claims_fin.drop(columns=['avg_hcp_cost_y', 'sd_hcp_cost'], inplace=True)

In [ ]:
## Rename "_x" to the normal name - 'avg_hcp_cost'
cpts_w_claims_fin.rename(index=str, columns={'avg_hcp_cost_x': 'avg_hcp_cost'}, inplace=True)

In [ ]:
## To ensure calculations are defined, replace nulls with 0.
cpts_w_claims_fin['UNITS_man'].fillna(0, inplace = True)

## Calculate ROI

For each spec/cpt code pair, we want to comapre the cost of reviewing the pair with the sum of dollars denied through that review. This is the "ROI" of reviewing. In cases where sum of denied dollars is greater than the cost of review, we recommend NOT auto-approving and continue to review. In cases where sum of denied dollars is less than the cost of review then we recommend auto-approving it. 

In [ ]:
## Calculate the total dollars denied for a pair
cpts_w_claims_fin['sum_cost_denied'] = cpts_w_claims_fin['is_den']*cpts_w_claims_fin['UNITS_man']*cpts_w_claims_fin['avg_hcp_cost']

In [ ]:
## Calc ROI for a pair
cpts_w_claims_fin['ROI'] = cpts_w_claims_fin['sum_cost_denied']/cpts_w_claims_fin['cost_to_review']

In [ ]:
## For groups where we don't know the average cost from 2018, the denominator of ROI is 0, and ROI is undefined. 
## Update the ROI for those to = 100 so they are NOT included in the dictionaries to auto-approve going forward.
cpts_w_claims_fin['ROI'] = np.where(cpts_w_claims_fin['avg_hcp_cost'].isnull(), 100, cpts_w_claims_fin['ROI'])

In [ ]:
## For groups that were auto-approved at 100%, the denominator of ROI is 0, and ROI is undefined. 
## Update the ROI for those to = 0 so they are included in the dictionaries to auto-approve going forward.
cpts_w_claims_fin['ROI'] = np.where(cpts_w_claims_fin['ROI'].isnull(), 0.01, cpts_w_claims_fin['ROI'])
        

In [ ]:
## Create a flag based on ROI indicated whether the analytics recommend a pair to be auto-approved
cpts_w_claims_fin['fin_aa_rec'] = np.where(cpts_w_claims_fin['ROI']<1, 1, 0)

In [ ]:
## QA step - Check that this equals G&A
cpts_w_claims_fin['cost_to_review'].sum() == dept_ga

In [ ]:
cpts_w_claims_fin_last3 = cpts_w_claims_fin[cpts_w_claims_fin['last3'] == 1]

In [ ]:
cpts_w_claims_fin_prior3 = cpts_w_claims_fin[cpts_w_claims_fin['last3'] == 0]

In [ ]:
cpts10 = pd.merge(cpts_w_claims_fin_last3, cpts_w_claims_fin_prior3, on = ['Specialty', 'CPT_Code', 'is_PPL'], 
                                   suffixes=('', '_prior3'), how='outer')

In [ ]:
cpts10.drop(['last3', 'cnt_hcp_cost', 'cost_to_review', 'sum_cost_denied', 'last3_prior3', 'cnt_hcp_cost_prior3',
                               'cost_to_review_prior3', 'cnt_hcp_cost_prior3', 'avg_hcp_cost_prior3', 'sum_cost_denied_prior3'], axis=1
                              , inplace=True)

In [ ]:
## Calculate Increase in volume
cpts10['vol_increase'] = (cpts10['UNITS']/cpts10['UNITS_prior3']) - 1

In [ ]:
## Calculate increase in AA rate
cpts10['AA_rate_increase'] = cpts10['is_autoapp']-cpts10['is_autoapp_prior3']

In [ ]:
## Calculate increase in denial rate
cpts10['den_rate_increase'] = cpts10['is_den']-cpts10['is_den_prior3']

In [ ]:
cpts10 = cpts10.merge(cpt_desc, how='inner', on = 'CPT_Code')

In [ ]:
cpts11 = pd.merge(clinical_decisions_current, cpts10, on=['Specialty', 'CPT_Code', 'is_PPL'], how = 'outer')

In [ ]:
cpts11.columns

In [ ]:
cpts11['Reason'].fillna('DBA', inplace=True)

In [ ]:
cpts11['Reason'].unique()

In [ ]:
## flag retro statuses with 1 and 0 so they can be removed 
reason_categories = [
 (cpts11['Reason'] == 'Heavy Cap Vol'),
 (cpts11['Reason'] == 'Overutilization Concern'),
 (cpts11['Reason'] == 'Upcoding'),
 (cpts11['Reason'] == 'Cost Containment'),
 (cpts11['Reason'] == 'Inappropriate Location'),
 (cpts11['Reason'] == 'Medical Necessity Review'),
 (cpts11['Reason'] == 'RNL'),
 (cpts11['Reason'] == 'Unclassified Drug'),
 (cpts11['Reason'] == 'Override Pend'),
 (cpts11['Reason'] == 'DBA'),
 (cpts11['Reason'] == 'Pend Specialty'),
 (cpts11['Reason'] == 'Pend EPL'),
 (cpts11['Reason'] == 'Low Volume') 
  ]

reason_choices = ['M', 'M', 'M', 'do not AA', 'do not AA', 'do not AA', 'do not AA', 'do not AA', 'AA', 'AA', 'do not AA', 'do not AA', 'do not AA']
cpts11['reason_cat'] = np.select(reason_categories, reason_choices, default=0)

In [ ]:
## Reorder the columns to make easier to compare

cpts11 = cpts11[['Specialty', 'CPT_Code', 'Name', 'is_PPL', 'Decision_Source',
       'Reason', 'reason_cat', 'Decision_Maker', 'Decision_Date', 'Decision', 'UNITS',
       'UNITS_prior3', 'vol_increase', 'is_autoapp', 'is_autoapp_prior3',
       'AA_rate_increase', 'is_den', 'is_den_prior3', 'den_rate_increase',
       'UNITS_man', 'UNITS_man_prior3', 'avg_hcp_cost', 'ROI', 'ROI_prior3',
       'fin_aa_rec', 'fin_aa_rec_prior3']]

In [ ]:
cpts11.sort_values('UNITS', inplace=True, ascending=False)

In [ ]:
cpts11.head()

In [ ]:
# Give the filename you wish to save the file to
quarterly_comparison = '../Data/Outputs/quarterly_comparison.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(quarterly_comparison)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    cpts11.to_excel(quarterly_comparison, index=False, freeze_panes=(1,0))
    print('The file did not exist, so created it.')
else:
    print('WARNING: This file already exists!')